# Continuous Control: Unity ML Reacher environment
---

##  Solving Unity-ML Reacher environment using the Proximal Policy Optimization (PPO) algorithm

Proximal Policy Optimization (PPO) is a reinforcement learning algorithm designed to address the challenges of policy-based optimization. It operates by updating the policy in a manner that doesn't change it too drastically, using a clipped objective function to prevent large policy updates. This ensures stable and efficient learning by avoiding overly aggressive updates that can harm the agent's performance.
Also, when calculating the optimizing function (objective), we'll use the advantage calculation, instead of using the expected future return for each state.

There are a few considerations for this implementation of the PPO algorithm that should be highlighted:

1. ThisPPO often operates within an Actor-Critic framework. The "Actor" is the policy being optimized and the "Critic" estimates the value function, V(s). The advantage is used to coordinate updates between these two components. The critic's value function estimates help in computing the advantage, which in turn is used to update the actor.
2. 

### 1. The Environment


## Reacher Environment Description:

In the Reacher environment, an agent controls a double-jointed arm. The primary objective of the agent is to move its hand (the end of the arm) to a target location and keep it there. The target can move over time, which presents a continuous tracking challenge for the agent. The agent receives a reward based on the distance between the hand and the target - the closer the hand is to the target, the higher the reward.

### Multi-Agent Variant:

The multi-agent version of the Reacher environment contains multiple double-jointed arms, with each arm controlled by an independent agent. All agents share the same environment, but each agent is trained individually to control its own arm. The presence of multiple agents can make the task more challenging because the agents must learn not just to reach their own individual targets but also to potentially coordinate and avoid colliding with other agents.

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
# Load the environment
from unityagents import UnityEnvironment
import time
# env = UnityEnvironment(file_name='../../unity_ml_envs/Reacher_Windows_x86_64/Reacher.exe')
env = UnityEnvironment(file_name='../PPO-Reacher_UnityML/Reacher_Windows_x86_64/Reacher.exe')
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [2]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])
print(f"Size of state: {states[0].shape}")

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]
Size of state: (33,)


### 3. Setup Hyperparameters

In [3]:
# Training Hyperparameters
EPISODES = 1000         # Number of episodes to train for
# MAX_T = 2048          # Max length of trajectory
MAX_T = 1000            # Max length of trajectory
SGD_EPOCHS = 4          # Number of gradient descent steps per batch of experiences
BATCH_SIZE = 32         # minibatch size
BETA = 0.01             # entropy regularization parameter
GRADIENT_CLIP = 5       # gradient clipping parameter

# optimizer parameters
LR = 3e-4               # learning rate
EPSILON = 1e-5          # optimizer epsilon

# PPO parameters
GAMMA = 0.99            # Discount factor
TAU = 0.95              # GAE parameter
PPO_CLIP_EPSILON = 0.2  # ppo clip parameter


### 4. Agent

For this PPO solution, we are going to use an Actor-Critic architecture, in which the actor chooses the action directly, while the critic evaluates each state for calculating the TD error estimate in the advantage calculation.

In [4]:
# Agent and models
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


class Actor(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=64):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, action_size)
        
    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return F.tanh(self.fc3(x))
    
    
class Critic(nn.Module):
    def __init__(self, state_size, value_size=1, hidden_size=64):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, value_size)
        
    def forward(self, states):
        x = F.relu(self.fc1(states))
        x = F.relu(self.fc2(x))
        return self.fc3(x)
    
    
class ActorCritic(nn.Module):
    def __init__(self, state_size, action_size, value_size=1, hidden_size=64, std=0.0):
        super(ActorCritic, self).__init__()
        self.actor = Actor(state_size, action_size, hidden_size)
        self.critic = Critic(state_size, value_size, hidden_size)
        
        self.log_std = nn.Parameter(torch.ones(1, action_size)*std)
        
    def forward(self, states): # TODO: LEARN WHAT THE FUCK THIS DOES
        obs = torch.FloatTensor(states)
        
        # Critic
        values = self.critic(obs)
        
        # Actor
        mu = self.actor(obs)
        std = self.log_std.exp().expand_as(mu)
        dist = torch.distributions.Normal(mu, std)
        
        return dist, values
    
    
class Agent():
    def __init__(self, num_agents, state_size, action_size):
        self.num_agents = num_agents
        self.state_size = state_size
        self.action_size = action_size
        self.model = ActorCritic(state_size, action_size, value_size=1)
        self.optimizer = optim.Adam(self.model.parameters(), lr=LR, eps=EPSILON)
        self.model.train()
        
    def act(self, states):
        """Remember: states are state vectors for each agent
        It is used when collecting trajectories
        """
        dist, values = self.model(states)           # pass the state trough the network and get a distribution over actions and the value of the state
        actions = dist.sample()                     # sample an action from the distribution
        log_probs = dist.log_prob(actions)          # calculate the log probability of that action
        log_probs = log_probs.sum(-1).unsqueeze(-1) # sum the log probabilities of all actions taken (in case of multiple actions) and reshape to (batch_size, 1)
        
        return actions, log_probs, values

    
    def learn(self, states, actions, log_probs_old, returns, advantages, sgd_epochs=4):
        """ Performs a learning step given a batch of experiences
        
        Remmeber: in the PPO algorithm, we perform SGD_episodes (usually 4) weights update steps per batch
        using the proximal policy ratio clipped objective function
        """        

        num_batches = states.size(0) // BATCH_SIZE
        for i in range(sgd_epochs):
            batch_count = 0
            batch_ind = 0
            for i in range(num_batches):
                sampled_states = states[batch_ind:batch_ind+BATCH_SIZE, :]
                sampled_actions = actions[batch_ind:batch_ind+BATCH_SIZE, :]
                sampled_log_probs_old = log_probs_old[batch_ind:batch_ind+BATCH_SIZE, :]
                sampled_returns = returns[batch_ind:batch_ind+BATCH_SIZE, :]
                sampled_advantages = advantages[batch_ind:batch_ind+BATCH_SIZE, :]
                
                L = ppo_loss(self.model, sampled_states, sampled_actions, sampled_log_probs_old, sampled_returns, sampled_advantages)
                
                self.optimizer.zero_grad()
                (L).backward()
                nn.utils.clip_grad_norm_(self.model.parameters(), GRADIENT_CLIP)
                self.optimizer.step()
                
                batch_ind += BATCH_SIZE
                batch_count += 1

## Training

### 4. Loss function: PPO Clipped Surrogate Function with value loss (critic)

**PPO ratio**: By taking the ratio between the last policy (the one used to colelct the trajectory) and the current one (updated in the last SGD epoch), we ensure that the policy doesn't change too drastically, preventing potential instabilities in learning.

**Clipping:** PPO's clipping mechanism ensures that the policy doesn't get updated too much in one step, preventing disruptive updates and ensuring smoother learning.

**Entropy Bonus:** Including the entropy bonus ensures sufficient exploration, which is essential for finding optimal or near-optimal policies, especially in environments with deceptive local optima.

**Value Loss:** Using a critic to estimate the value function can reduce the variance in policy updates, leading to faster and more stable convergence. The value loss ensures that this critic provides accurate value estimates.

In [5]:
# Loss function. NOT INTEGRATED YET
def ppo_loss(model, states, actions, log_probs_old, returns, advantages):
    dist, values = model(states)
    
    log_probs = dist.log_prob(actions)
    log_probs = torch.sum(log_probs, dim=1, keepdim=True)
    entropy = dist.entropy().mean()
    
    # r(θ) =  π(a|s) / π_old(a|s)
    ratio = (log_probs - log_probs_old).exp()
    
    # Surrogate Objctive : L_CPI(θ) = r(θ) * A
    obj = ratio * advantages
    
    # clip ( r(θ), 1-Ɛ, 1+Ɛ )*A
    obj_clipped = ratio.clamp(1.0 - PPO_CLIP_EPSILON, 1.0 + PPO_CLIP_EPSILON) * advantages
    
    # L_CLIP(θ) = E { min[ r(θ)A, clip ( r(θ), 1-Ɛ, 1+Ɛ )*A ] - β * KL }
    policy_loss = -torch.min(obj, obj_clipped).mean(0) - BETA * entropy.mean()
    
    # L_VF(θ) = ( V(s) - V_t )^2
    value_loss = 0.5 * (returns - values).pow(2).mean()
    
    return policy_loss + value_loss

### 5. Advantage Calculation

Why would we use the advantage instead of using the raw future rewards when calculating the objective function?

**Variance Reduction:** Raw future rewards are an "infinite n-step" bootstrap, and use full rollouts when calculating, which means that is has high variance. This is because the rewards in many environments can be stochastic, leading to different return values even for the same state-action pairs.
The advantage function, A(s,a)=Q(s,a)−V(s), subtracts the value of being in state s from the Q-value, resulting in a measure that tells us how much better an action a is than the average action in that state. By centering our updates around this advantage, we're essentially normalizing our updates, leading to more stable learning.

**Bias-Variance Trade-off:** In reinforcement learning, there's a trade-off between bias and variance. Using raw future rewards can have low bias but high variance. On the other hand, using just the value function (like a TD error) might have lower variance but higher bias. The advantage aims to strike a balance by considering both the current estimate (value function) and the sampled return.

**Better Signal for Learning:** The advantage provides a more direct signal about the quality of an action. If an action's advantage is positive, it means the action is better than what's typically expected in that state. If it's negative, it means the action is worse. This kind of differentiation might be lost when using raw future rewards, especially in states where all actions lead to similarly high rewards.



In [6]:
def calculate_advantages(rollout, returns, num_agents):
    """ Given a rollout, calculates the advantages for each state """
    num_steps = len(rollout) - 1
    processed_rollout = [None] * num_steps
    advantages = torch.zeros((num_agents, 1))

    for i in reversed(range(num_steps)):
        states, value, actions, log_probs, rewards, dones = map(lambda x: torch.Tensor(x), rollout[i])
        next_value = rollout[i + 1][1]

        dones = dones.unsqueeze(1)
        rewards = rewards.unsqueeze(1)

        # Compute the updated returns
        returns = rewards + GAMMA * dones * returns

        # Compute temporal difference error
        td_error = rewards + GAMMA * dones * next_value.detach() - value.detach()
        
        advantages = advantages * TAU * GAMMA * dones + td_error
        processed_rollout[i] = [states, actions, log_probs, returns, advantages]

    # Concatenate along the appropriate dimension
    states, actions, log_probs_old, returns, advantages = map(lambda x: torch.cat(x, dim=0), zip(*processed_rollout))
    
    # Normalize advantages
    advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
    
    return states, actions, log_probs_old, returns, advantages

### 6. Collect trajectories and train

In [13]:
import numpy as np
from collections import deque
import torch
from src.utils import test_agent
import os

def collect_trajectories(env, brain_name, agent, max_t):
    env_info = env.reset(train_mode=True)[brain_name]
    num_agents = len(env_info.agents)
    states = env_info.vector_observations
        
    rollout = []
    agents_rewards = np.zeros(num_agents)
    episode_rewards = []

    for _ in range(max_t):
        actions, log_probs, values = agent.act(states)
        env_info = env.step(actions.cpu().detach().numpy())[brain_name]
        next_states = env_info.vector_observations
        rewards = env_info.rewards 
        dones = np.array([1 if t else 0 for t in env_info.local_done])
        agents_rewards += rewards

        for j, done in enumerate(dones):
            if dones[j]:
                episode_rewards.append(agents_rewards[j])
                agents_rewards[j] = 0

        rollout.append([states, values.detach(), actions.detach(), log_probs.detach(), rewards, 1 - dones])

        states = next_states

    pending_value = agent.model(states)[-1]
    returns = pending_value.detach() 
    rollout.append([states, pending_value, None, None, None, None])
    
    return rollout, returns, episode_rewards, np.mean(episode_rewards)


def train(env, brain_name, agent, num_agents, n_episodes, max_t, run_name="testing_01"):
    print(f"Starting training...")
    env.info = env.reset(train_mode = True)[brain_name]
    all_scores = []
    all_scores_window = deque(maxlen=100)
    best_so_far = 0.0
        
    for i_episode in range(n_episodes):
        # Each iteration, N parallel actors collect T time steps of data
        rollout, returns, _, _ = collect_trajectories(env, brain_name, agent, max_t)
        
        states, actions, log_probs_old, returns, advantages = calculate_advantages(rollout, returns, num_agents)
        # print(f"States: {states.shape}. Actions: {actions.shape}. Log_probs_old: {log_probs_old.shape}. Returns: {returns.shape}. Advantages: {advantages.shape}")
        agent.learn(states, actions, log_probs_old, returns, advantages)
        
        test_mean_reward = test_agent(env, agent, brain_name)

        all_scores.append(test_mean_reward)
        all_scores_window.append(test_mean_reward)

        if np.mean(all_scores_window) > best_so_far:
            if not os.path.isdir(f"./ckpt/{run_name}/"):
                os.mkdir(f"./ckpt/{run_name}/")
            torch.save(agent.model.state_dict(), f"./ckpt/{run_name}/ppo_checkpoint_{np.mean(all_scores_window)}.ckpt")
            best_so_far = np.mean(all_scores_window)
            if np.mean(all_scores_window) > 30:
                
                print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(all_scores_window)))
                # break       
        
        print('Episode {}, Total score this episode: {}, Last {} average: {}'.format(i_episode + 1, test_mean_reward, min(i_episode + 1, 100), np.mean(all_scores_window)) )

In [11]:

env_info = env.reset(train_mode=True)[brain_name]
time.sleep(2)

# Environment variables
num_agents = len(env_info.agents)
state_size = env_info.vector_observations.shape[1]
action_size = brain.vector_action_space_size

# Instantiate the agent
agent = Agent(num_agents, state_size, action_size)

In [14]:
# Train the agent
train(env, brain_name, agent, num_agents, EPISODES, MAX_T)
env.close()

Starting training...
Episode 1, Total score this episode: 0.3714999916963279, Last 1 average: 0.3714999916963279
Episode 2, Total score this episode: 0.363999991863966, Last 2 average: 0.36774999178014695


KeyboardInterrupt: 

## Test trained agent

In [14]:
from src.utils import load_trained_agent
agent = load_trained_agent(env, "./ckpt/run_2/ppo_checkpoint_36.132574192374015.pth")
test_agent(env, agent, brain_name)

38.60449913712218